In [1]:
try:
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    import datetime as dt
    pd.set_option('display.max_columns',None)
    print('libraries are found')
except:
    print('Some libraries need to be installed')

libraries are found


In [2]:
import pandas as pd
# file1 = '../data/train/data_2022-03-29_01-26-13_PM.h5'
# file2 = '../data/train/data_2022-05-14_10-07-26_AM.h5'

file = '/workspace/luca/Tabular/e2e/data/data_2022-05-14_10-07-26_AM.h5'
df = pd.read_hdf(file, key='data')

FileNotFoundError: File /workspace/luca/Tabular/e2e/data/data_2022-05-14_10-07-26_AM.h5 does not exist

In [ ]:
df.shape

In [ ]:
## remove columns which contain more than 50% of null values
def treat_null_values(df):
    
    df = df.dropna(thresh=df.shape[0]*0.5, how='all',axis=1) # -3
    df = df.dropna(axis=0,subset=['RECEIPT_DATE','BOOKED_DATE'])
    df = df[df['RECEIPT_DATE']>df['BOOKED_DATE']] # false
       
    return df

df = treat_null_values(df)
df.shape

In [ ]:
df = df.set_index('PO_STRING',drop=True)

In [ ]:
# optional function
#  This funtion is to analyse the different types of columns # optional
def analyze_data_types(df):
    date_columns_df = df.select_dtypes(include=['datetime64'])
    object_columns_df = df.select_dtypes(include=['object'])
    numerics= [ 'int64','float64']
    numeric_columns_df = df.select_dtypes(include = numerics)
    
    return date_columns_df, object_columns_df,numeric_columns_df

date_df, objects_df,numeric_df=analyze_data_types(df)

In [ ]:
#df.info()

In [ ]:
def drop_features(df,cols_to_drop):
    df['diff_SO_LINE_FIRST_SO_LINE_LAST'] = df['SO_LINE_LAST'] - df['SO_LINE_FIRST']

    df.drop(cols_to_drop,axis=1,inplace=True)
    
    return df



In [ ]:
columns_to_remove = ['ORG_ID','ATTRIBUTE_NUMBER1','ATTRIBUTE_NUMBER2','ATTRIBUTE_NUMBER6','ATTRIBUTE_NUMBER7','ATTRIBUTE_NUMBER8',
                    'UNIT_PRICE_USD','SO_LINE_FIRST','SO_LINE_LAST']

#objects_columns can be removed except 'TRANSPORTATION_LEAD_TIME'

objects_df_cols=list(objects_df.columns)
objects_df_cols.remove('TRANSPORTATION_LEAD_TIME')
cols_to_remove = columns_to_remove + objects_df_cols

In [ ]:
len(cols_to_remove)

In [ ]:
df = drop_features(df,cols_to_remove)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df=df.dropna()

In [ ]:
df.shape

In [ ]:
# function to re-engineer the dates
def difference_dates(df,col1,col2):
            
    df['diff_'+col1+col2]=(df[col1]-df[col2]).dt.days
    df.drop([col1,col2],axis=1,inplace = True)
    
    return df
# call function on the required features    
df = difference_dates(df,'RTS_','RTS_FINAL')
df = difference_dates(df,'SO_CWD','SO_CREATION_DATE')
df = difference_dates(df,'EOQ_CONTR','EOQ_REQUEST')
df = difference_dates(df,'CONTRACTUAL_DATE','FULFILLMENT_CONTRACTUAL_DATE')
#df = difference_dates(df,'PO_FIRST_APPROVED_DATE','POR_APPROVED_DATE')
df = difference_dates(df,'RECEIPT_DATE','BOOKED_DATE') 
df = difference_dates(df,'PROMISED_DATE_LAST','PROMISED_DATE_FIRST')


# since most of the rows are same as NEED_BY_DATE','NEED_DATE_FIRST', 'NEED_DATE_LAST'
# 'NEED_BY_DATE' has same values as 'SUP_PO_NEED_BY_DATE'
# instead of these 3 features only one feature can be used and name it 'NEED_DATE_'
# 'DATE_WHEN_NEED_REV'
df['NEED_DATE_'] = df['NEED_BY_DATE']
df.drop(['NEED_DATE_FIRST', 'NEED_DATE_LAST','SUP_PO_NEED_BY_DATE','DATE_WHEN_NEED_REV',
         'NEED_BY_DATE'],axis=1,inplace=True)



# both SCHEDULE_SHIP_DATE and 'LINE_REQUEST_DATE' are greater than NEED_DATE so take the diff
df['diff_SCHEDULE_SHIP_DATE_NEED_DATE_'] = (df['SCHEDULE_SHIP_DATE']-df['NEED_DATE_']).dt.days
df['diff_LINE_REQUEST_DATE_NEED_DATE_'] = (df['LINE_REQUEST_DATE']-df['NEED_DATE_']).dt.days

df.drop(['LINE_REQUEST_DATE','SCHEDULE_SHIP_DATE','PO_FIRST_APPROVED_DATE'],axis=1,inplace=True)


In [ ]:
df.info()